## function of create declearitive sentences

In [1]:
import pandas as pd
import spacy
import random
from sentence_transformers import SentenceTransformer, util
import torch
from tqdm import tqdm
import json
nlp = spacy.load("en_core_web_sm")
def create_declearitive(sent):
    sent = sent.lower()
    doc = nlp(sent)
    if str(doc[0]) == "because" or str(doc[0]) == "as" or str(doc[0]) == "since" or str(doc[0]) == "so":
        word = str(doc[0]) if sent.lower().find("so that") == -1 else "so that"
        return_sent = sent.replace(word,"",1)[1:].lower()
    elif str(doc[0]) == "to":
        return_sent = "The car wants " + sent.lower()
    elif str(doc[0]) == "at" or str(doc[0]) == "for" or str(doc[0]) == "due":
        word = "due to" if str(doc[0]) == "due" else str(doc[0])
        return_sent = "There is" + sent.lower().replace(word, "", 1)
    elif str(doc[0]) == "while":
        if doc[1].pos_ == "PRON":
            return_sent = sent.replace("while","",1)[1:].lower()
        else:
            return_sent = sent.replace("while","the car is",1).lower()
    else:
        return_sent = sent
    return_sent = return_sent.lower().replace("it", "the car", 1).replace("it's", "the car is", 1)
    return_sent = return_sent[0].upper() + return_sent[1:].lower()
    return_sent = return_sent if return_sent[-1] == "." else return_sent + "."
    return return_sent
create_declearitive("while it waits for a pedestrian to cross the road.")

'The car waits for a pedestrian to cross the road.'

## get "dataset" with sentence embeddings

In [3]:
df = pd.read_csv("../../data/bdd/raw.csv")
df = df.fillna("")
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device="cuda")
justi_names = [col for col in df.columns if "justi" in col]
dataset = []
premises = []
hypothesises = []
pbar = tqdm(total = df.shape[0], ncols = 100)
for index, row in df.iterrows():
    for j_colname in justi_names:
        a_colname = j_colname.replace("justification", "action")
        i += 1
        if row[j_colname] != "" and row[j_colname] != None and row[j_colname].find("unknown") != 0 and len(row[a_colname])>5 and len(row[j_colname])>5:
            
            premise = create_declearitive(row[j_colname])
            hypothesis = row[a_colname]
            premises.append(premise)
            hypothesises.append(hypothesis)
            data = {}
            data["premise"] = premise
            data["hypothesis"] = hypothesis
            data["indice"] = (index, j_colname)
            hypothesis_embed = model.encode(hypothesis)
            premise_embed = model.encode(premise)
            data["hembed"] = hypothesis_embed
            data["pembed"] = premise_embed
            dataset.append(data)
    pbar.update()

  0%|                                                                     | 0/12997 [00:28<?, ?it/s]


194955
26523


## Remove the too similar premises

In [12]:
embeds = []
new_dataset = []
i = -1
for data in tqdm(dataset):
    for i, e in enumerate(embeds):
        if util.pytorch_cos_sim(e, data["pembed"]) > 0.9:
            break
    if i == len(embeds) - 1:
        new_dataset.append(data)
        embeds.append(data["pembed"])
dataset = new_dataset

100%|██████████| 4800/4800 [08:19<00:00,  9.61it/s]███████████| 12997/12997 [06:40<00:00, 41.40it/s]


## Build two multiple choice datasets, sampled by cause and effect

In [17]:
test_datas1 = []
test_datas2 = []

l = len(dataset)
for data in tqdm(dataset, ncols=100):
    hypothesis_embed = data["hembed"]
    distractors = []
    embeds = []
    embeds.append(hypothesis_embed)
    while len(distractors) < 2:
        num = random.sample(range(0,len(dataset)),1)[0]
        distractor_data = dataset[num]
        distractor = distractor_data["hypothesis"]
        if distractor not in distractors:
            distractor_embed = distractor_data["hembed"]
            if_add = 1
            for embed in embeds:
                if util.pytorch_cos_sim(embed, distractor_embed) > 0.4:
                    if_add = 0
            if if_add == 1:
                distractors.append(distractor)
                embeds.append(distractor_embed)
    test_data = {}
    test_data["c"] = data["premise"]
    test_data["e"] = [distractors[0], distractors[1], data["hypothesis"]]
    test_data["answer"] = 2
    test_datas1.append(test_data)

for data in tqdm(dataset, ncols=100):
    premise_embed = data["pembed"]
    distractors = []
    embeds = []
    embeds.append(premise_embed)
    while len(distractors) < 2:
        num = random.sample(range(0,len(dataset)), 1)[0]
        distractor_data = dataset[num]
        distractor = distractor_data["premise"]
        if distractor not in distractors:
            distractor_embed = distractor_data["pembed"]
            if_add = 1
            for embed in embeds:
                if util.pytorch_cos_sim(embed, distractor_embed) > 0.4:
                    if_add = 0
            if if_add == 1:
                distractors.append(distractor)
                embeds.append(distractor_embed)
    test_data = {}
    test_data["c"] = [distractors[0], distractors[1], data["premise"]]
    test_data["e"] = data["hypothesis"]
    test_data["answer"] = 2
    test_datas2.append(test_data)

100%|██████████████████████████████████████████████████████████| 3139/3139 [00:03<00:00, 882.37it/s]


In [14]:
with open("../../data/bdd/e.jsonl", "w")as f:
    for i, data in enumerate(test_datas1):
        data["id"] = i
        f.write(json.dumps(data))
        f.write("\n")
with open("../../data/bdd/c.jsonl", "w")as f:
    for i, data in enumerate(test_datas2):
        data["id"] = i
        f.write(json.dumps(data))
        f.write("\n")

## Test for deciding the threshold

In [8]:
from sentence_transformers import SentenceTransformer, util
ori = "The car slows to a stop"
sentences = ["The vehicle is driving forward",
             "The vehicle is cruising along the road",
             "The car is driving forward",
             "The car is cruising along the road",
             "The car is steering to the right",
             "The car makes a right hand turn with its windshield wipers on",
             "The vehicle is motionless", 
             "The vehicle is not moving",
             "The car is motionless", 
             "The car is not moving"]

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
oriembed = model.encode(ori)
embeddings = model.encode(sentences)
for e in embeddings:
    print(util.cos_sim(oriembed, e))

tensor([[0.4311]])
tensor([[0.2769]])
tensor([[0.4761]])
tensor([[0.3100]])
tensor([[0.3847]])
tensor([[0.3753]])
tensor([[0.5299]])
tensor([[0.5334]])
tensor([[0.6020]])
tensor([[0.6243]])
